## Imports and global parameters

In [1]:
import os
import re

import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import librosa
import noisereduce
import librosa.display
import IPython.display as ipd

from sklearn.utils import shuffle
from PIL import Image

import tensorflow as tf
from tensorflow.keras.applications import ResNet50

import keras
from keras import layers
from keras.utils import to_categorical
from keras.models import Model, load_model
#from keras.initializers import glorot_uniform
from keras.layers import Input, Dropout, Add, Dense
from keras.layers import AveragePooling2D, Reshape, Activation
from keras.layers import BatchNormalization, Flatten, Conv1D

import utils

# Global vars
RANDOM_SEED = 1337
SAMPLE_RATE = 32000
SIGNAL_LENGTH = 5 # seconds
SPEC_SHAPE = (48, 128) # height x width
INPUT_SHAPE = SPEC_SHAPE + (3,)
FMIN = 500
FMAX = 12500
MAX_AUDIO_FILES = 1500

input_dir = 'basic_test/birds/'
output_dir = 'basic_test/melspectrogram_dataset/'
model_dir = 'basic_test/'
soundscape_dir = 'basic_test/'

Hello Kaare
utils


## Get data

### Select data
Find all bird species with more than 200 entries in the training data and save that data as TRAIN and LABELS

In [2]:
# Load metadata file
train = pd.read_csv('train_metadata.csv',)

# Limit the number of training samples and classes
# First, only use high quality samples
train = train.query('rating>=4')

# Second, assume that birds with the most training samples are also the most common
# A species needs at least 200 recordings with a rating above 4 to be considered common
birds_count = {}
for bird_species, count in zip(train.primary_label.unique(), 
                               train.groupby('primary_label')['primary_label'].count().values):
    birds_count[bird_species] = count
most_represented_birds = [key for key,value in birds_count.items() if value >= 200] 

TRAIN = train.query('primary_label in @most_represented_birds')
LABELS = sorted(TRAIN.primary_label.unique())

# Let's see how many species and samples we have left
print('NUMBER OF SPECIES IN TRAIN DATA:', len(LABELS))
print('NUMBER OF SAMPLES IN TRAIN DATA:', len(TRAIN))
print('LABELS:', most_represented_birds)

NUMBER OF SPECIES IN TRAIN DATA: 27
NUMBER OF SAMPLES IN TRAIN DATA: 8548
LABELS: ['amerob', 'barswa', 'bewwre', 'blujay', 'bncfly', 'carwre', 'compau', 'comrav', 'comyel', 'eursta', 'gbwwre1', 'grekis', 'houspa', 'houwre', 'mallar3', 'norcar', 'normoc', 'redcro', 'rewbla', 'roahaw', 'rubpep1', 'rucspa1', 'sonspa', 'spotow', 'wbwwre1', 'wesmea', 'yeofly1']


In [3]:
# Shuffle the training data and limit the number of audio files to MAX_AUDIO_FILES
TRAIN = shuffle(TRAIN, random_state=RANDOM_SEED)[:MAX_AUDIO_FILES]

print('FINAL NUMBER OF AUDIO FILES IN TRAINING DATA:', len(TRAIN))

FINAL NUMBER OF AUDIO FILES IN TRAINING DATA: 1500


### Load spectograms
Run the load function

In [4]:
# Parse audio files and extract training samples
samples = []
with tqdm(total=len(TRAIN)) as pbar:
    for idx, row in TRAIN.iterrows():
        pbar.update(1)
        
        if row.primary_label in most_represented_birds:
            audio_file_path = os.path.join(input_dir, row.primary_label, row.filename)
            samples += utils.get_spectrograms(audio_file_path, row.primary_label, output_dir, SAMPLE_RATE, SIGNAL_LENGTH, SPEC_SHAPE, FMIN, FMAX)
            
TRAIN_SPECS = shuffle(samples, random_state=RANDOM_SEED)
print('SUCCESSFULLY EXTRACTED {} SPECTROGRAMS'.format(len(TRAIN_SPECS)))

100%|██████████| 1500/1500 [01:05<00:00, 22.93it/s]

SUCCESSFULLY EXTRACTED 4157 SPECTROGRAMS


### Load data
Load spectograms and normalize, and add to a stack of 4154 sound bites, of 48 (MEL resolution) x 128 (time resolution) pixels. Labels are 4154 one-hot vectors of length 27 (number of bird types)

In [5]:
# Parse all samples and add spectrograms into train data, primary_labels into label data
train_specs, train_labels = [], []
with tqdm(total=len(TRAIN_SPECS)) as pbar:
    for path in TRAIN_SPECS:
        pbar.update(1)

        # Open image
        spec = Image.open(path)

        # Convert to numpy array
        spec = np.array(spec, dtype='float32')
        
        # Normalize between 0.0 and 1.0
        # and exclude samples with nan 
        spec -= spec.min()
        spec /= spec.max()
        if not spec.max() == 1.0 or not spec.min() == 0.0:
            continue

        # Add channel axis to 2D array
        spec = np.expand_dims(spec, -1)

        # Add new dimension for batch size
        spec = np.expand_dims(spec, 0)

        # Add to train data
        if len(train_specs) == 0:
            train_specs = spec
        else:
            train_specs = np.vstack((train_specs, spec))

        # Add to label data
        target = np.zeros((len(LABELS)), dtype='float32')
        bird = path.replace(os.sep, '/').split('/')[-2]
        target[LABELS.index(bird)] = 1.0
        if len(train_labels) == 0:
            train_labels = target
        else:
            train_labels = np.vstack((train_labels, target))

100%|██████████| 4157/4157 [02:16<00:00, 30.45it/s]


### Repeat in channel dimension and preprocess

In [6]:
train_input = tf.keras.applications.resnet.preprocess_input(np.repeat(train_specs, 3, axis=3))

## Create model

### Load base model

In [39]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(INPUT_SHAPE)))
#baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

In [40]:
baseModel.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 48, 128, 3)] 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 54, 134, 3)   0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 24, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 24, 64, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

### Create a new head

In [41]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(2, 4))(headModel)
#headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(LABELS), activation="softmax")(headModel)

### Add to model

In [42]:
model = Model(inputs=baseModel.input, outputs=headModel)
model.summary()

for layer in baseModel.layers[18:]:
	layer.trainable = False

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 48, 128, 3)] 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 54, 134, 3)   0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 24, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 24, 64, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________

In [43]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01),
              metrics=['accuracy'])

In [46]:
# Add callbacks to reduce the learning rate if needed, early stopping, and checkpoint saving
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                  patience=2, 
                                                  verbose=1, 
                                                  factor=0.5),
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                              verbose=1,
                                              patience=5),
             tf.keras.callbacks.ModelCheckpoint(filepath=model_dir + 'best_model.h5', 
                                                monitor='val_loss',
                                                verbose=0,
                                                save_best_only=True)]

callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                  patience=2, 
                                                  verbose=1, 
                                                  factor=0.5),
             tf.keras.callbacks.ModelCheckpoint(filepath=model_dir + 'best_model.h5', 
                                                monitor='val_loss',
                                                verbose=0,
                                                save_best_only=True)]

## Train model

In [47]:
 # Let's train the model for a few epochs
model.fit(train_input, 
          train_labels,
          batch_size=32,
          validation_split=0.2,
          callbacks=callbacks,
          epochs=25)

Epoch 1/25
104/104 [==============================] - 241s 2s/step - loss: 2.5601 - accuracy: 0.2615 - val_loss: 3.7185 - val_accuracy: 0.0325
Epoch 2/25
104/104 [==============================] - 245s 2s/step - loss: 2.4907 - accuracy: 0.2844 - val_loss: 4.1900 - val_accuracy: 0.0120
Epoch 3/25
104/104 [==============================] - ETA: 0s - loss: 2.4248 - accuracy: 0.3130
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
104/104 [==============================] - 248s 2s/step - loss: 2.4248 - accuracy: 0.3130 - val_loss: 3.8828 - val_accuracy: 0.0662
Epoch 4/25
104/104 [==============================] - 247s 2s/step - loss: 2.3212 - accuracy: 0.3376 - val_loss: 6.3576 - val_accuracy: 0.0698
Epoch 5/25
104/104 [==============================] - ETA: 0s - loss: 2.2934 - accuracy: 0.3497
Epoch 00005: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
104/104 [==============================] - 247s 2s/step - loss: 2.2934 - accuracy: 0.349